<a href="https://colab.research.google.com/github/BadPrograms/ICA/blob/main/unmixer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from numba import njit
import random
import matplotlib.pyplot as plt

scaler = StandardScaler(with_std=False)
@njit
def logistic(u):
    return 1/(1+np.exp(-u))

In [5]:
def whiten(X,fudge=1E-10):

   # the matrix X should be observations-by-components

   # get the covariance matrix
   Xcov = np.dot(X.T,X)
   #print(Xcov.shape)
   X=scaler.fit_transform(X)
   # eigenvalue decomposition of the covariance matrix
   d, V = np.linalg.eigh(Xcov)

   # a fudge factor can be used so that eigenvectors associated with
   # small eigenvalues do not get overamplified.
   D = np.diag(1. / np.sqrt(d+fudge))

   # whitening matrix
   W = np.dot(np.dot(V, D), V.T)
   #print(W.shape)
   # multiply by the whitening matrix
   
   X_white= 2*np.dot(W,X.T)
   return X_white


In [6]:
@njit
def ICA2(dt,iterations=2000,learningrate=0.1):
    W=np.random.rand(len(dt[0,:]),len(dt[0,:]))
    bias=np.random.rand(len(dt[0,:]))
    print(W.shape)
    print(len(dt))
    for _ in range(1,iterations):
      a=10**(int(_/1000)+1)
      print(_)
      for o in range(len(dt)):
        
        u=np.dot(W,dt[o,:]) +bias
        y=1/(1+np.exp(-u))
        #ycap=np.dot(y,np.exp(u)-1)
        dw=np.zeros((W.shape))
        dw=np.linalg.inv(W.T)+np.dot(np.ones(len(y))-2*y,dt[o,:])
        
        bias+=learningrate*(np.ones(len(y))-2*y)
        W+=learningrate/(a)*dw
        #print(W)
    return W

In [28]:
@njit
def ICA(dt,iterations=2000,learningrate=0.1):
    W=np.random.rand(len(dt[0,:]),len(dt[0,:]))
    #bias=np.random.rand(len(dt[0,:]))
    print(W.shape)
    print(len(dt))
    for _ in range(1,iterations):
      print(_)
      a=10**(int(_/1000)+1)
      for o in range(len(dt)):
       
        u=np.dot(W,dt[o,:]) #+bias
        y=1/(1+np.exp(-u))
        ycap=np.dot(y,np.exp(u)-1)
        dw=np.zeros((W.shape))
        dw=np.linalg.inv(W.T)+np.dot(np.ones(len(y))-2*ycap,dt[o,:])
        
        #bias+=learningrate*(np.ones(len(y))-2*y)
        W+=learningrate/(a)*dw
        #print(W)
    return W

In [8]:
import PIL as pil
from google.colab import files
import sys
import io
import wave
uploaded = files.upload()
#np.set_printoptions(threshold=sys.maxsize)
import matplotlib.pyplot as plt

In [9]:
from IPython.display import Audio, display
from scipy.io import wavfile as wf

#display(Audio(uploaded["beet.wav"], autoplay=True))


In [10]:
_1,s1=wf.read("beet.wav")

In [11]:
_2,s2=wf.read("beet9.wav")
_3,s3=wf.read("mike.wav")
_4,s4=wf.read("beet92.wav")
_5,s5=wf.read("street.wav")
print(s3.shape)

(191258,)


In [12]:
display(Audio(s1,rate=_1, autoplay=True))
display(Audio(s2,rate=_2, autoplay=True))


In [13]:
def mix_sources(sources, apply_noise=False):
    for i in range(len(sources)):
        max_val = np.max(sources[i])
        if(max_val > 1 or np.min(sources[i]) < 1):
            sources[i] = sources[i] / (max_val / 2) - 0.5
            
    mixture = np.c_[[source for source in sources]]
    
    if(apply_noise):
        mixture += 0.5 * np.random.normal(size=X.shape)
        
    return mixture
s1.shape[0]

210504

In [27]:
x = mix_sources([s3, s2[:s3.shape[0]],s1[:s3.shape[0]],s4[:s3.shape[0]],s5[:s3.shape[0]]])
A=np.random.rand(5,5)*100
print(np.linalg.inv(A))
x=np.dot(A,x)
print(f'Shape of s1: {s1.shape}, s2: {s1.shape}, Linear Mix: {x.shape}')
wf.write('./talk_and_music.wav', _2, x.mean(axis=0).astype(np.float32))

[[-0.07295884  0.04798139 -0.03377668  0.08591762 -0.05191   ]
 [ 0.07566784 -0.04604807  0.01988863 -0.06239969  0.04415022]
 [-0.02707426  0.00898647  0.0043019   0.02075004 -0.0089845 ]
 [-0.06054474  0.04393646 -0.00286865  0.03932148 -0.03956781]
 [ 0.07873075 -0.04393755  0.0211796  -0.08261996  0.05701098]]
Shape of s1: (210504,), s2: (210504,), Linear Mix: (5, 191258)


In [15]:
display(Audio(x[0],rate=_1, autoplay=True))
display(Audio(x[1],rate=_2, autoplay=True))

In [16]:
x_whiten=whiten(x.T)
print(x_whiten.shape)

(5, 191258)


In [17]:
display(Audio(x[0],rate=_1, autoplay=True))

In [18]:
a=ICA2(x_whiten.T)
a2=ICA(x_whiten.T)

<ipython-input-6-3a8b433e9b04>:11: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 2d, C), array(float64, 1d, A))
  u=np.dot(W,dt[o,:]) +bias
<ipython-input-6-3a8b433e9b04>:15: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, C), array(float64, 1d, A))
  dw=np.linalg.inv(W.T)+np.dot(np.ones(len(y))-2*y,dt[o,:])


(5, 5)
191258
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
27

In [19]:
b=np.dot(a,x)
print(a)
b2=np.dot(a2,x)

[[-10819.97822465 -10346.73975899 -12416.03010201 -11180.04201942
   -5452.40856191]
 [ -8861.76269958  -8561.16019611  -7150.82027346 -13902.26712871
  -11572.74288906]
 [ -9351.06862166 -14250.70644913  -6091.20359741  -7766.90852337
   -8702.45091983]
 [ -7985.35582386 -10075.55182161 -12597.5086373   -7185.41641862
  -12903.29704995]
 [-14349.47524794  -7213.324524    -7850.17002823  -7213.05184176
  -10008.131188  ]]


In [30]:
display(Audio(b[4],rate=_1, autoplay=True))
display(Audio(x[1],rate=_1, autoplay=True))

In [21]:
print(np.dot(a,A))

[[-21518.67065951 -34529.26155866 -14323.82430088 -24784.9561999
  -35249.99471236]
 [-21219.40180844 -38105.68040504 -16045.53770783 -26193.53405363
  -30771.45345278]
 [-21304.80486936 -34424.29902057 -14637.36775493 -20924.9551766
  -31074.63477776]
 [-22341.01593081 -34972.34338365 -16287.67096986 -23107.04660376
  -33100.95419988]
 [-19963.81273917 -32852.04862853 -17113.86313993 -22622.0596061
  -32044.69506096]]


In [22]:
from sklearn.decomposition import FastICA

In [23]:
a2=sk.decomposition.FastICA(x)

NameError: ignored

In [ ]:
# Compute ICA
ica =FastICA()
S_ = ica.fit(x_whiten.T).transform(x_whiten.T) 
print(S_)

In [ ]:
display(Audio(S_[:,1].T,rate=_1, autoplay=True))